<h2>Segmenting and Clustering Neighborhoods in Toronto</h2>

Import the Libraries we are going to use

In [95]:
import requests
from lxml import etree
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page = requests.get(wikipedia_link)
tree = etree.HTML(raw_wikipedia_page.text)

<ul>
    <li>Loop the table</li>
    <li>Assign Borough as Neighborhood when Neighborhood is Not assigned</li>
    <li>Concatenate Neighborhoods that have the same PostalCode</li>
</ul>

In [96]:
import pandas as pd

labels = ['PostalCode', 'Borough', 'Neighborhood']
listDict = {}
#Iterating the table
for table in tree.xpath('//table[@class="wikitable sortable"]'):
    for row in table.xpath('tbody/tr'):
        cols = row.xpath('td//text()')
        #Condition to skip the rows with Borough as Not assigned
        if len(cols)>0 and cols[1]!='Not assigned':
            try:
                cols.remove('\n')
            except:
                pass 
            cols[2] = cols[2].rstrip()
            #Assigning Borough as Neighborhood when Neighborhood is Not assigned
            if(cols[2]=='Not assigned'):
                cols[2] = cols[1]
            #Concatenating Neighborhoods that have the same PostalCode    
            if(cols[0] in listDict):
                value = list.get(cols[0])
                neighborhoodList = value[2]+', '+cols[2]
                value[2] = neighborhoodList
                listDict[cols[0]] = value
            else:
                listDict[cols[0]] = cols
 
valList = [ v for v in listDict.values() ]
df = pd.DataFrame.from_records(valList, columns=labels)
df.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park, Regent Park, Regent..."
3,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Man..."
4,M7A,Queen's Park,Queen's Park


Display the shape of the dataframe

In [86]:
df.shape

(103, 3)